In [24]:
"""
  Trabalho 2 - Métodos Númericos para Equações Diferenciais
  Aluno: Victor Luis Teixeira Reis - 202110049511
"""

from math import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def analytic_solution (x, t):
  serie_length = 1000
  partial_sum = 0

  ct = 0.607 # A condutividade térmica da água ct = 0,607 W/m ∙ °C para a água
  p = 1000 # A massa específica da água é de aproximadamente 1000 kg/m3
  cp = 4.18 # A capacidade térmica específica da água é de aproximadamente 4.18 joules por grama por grau Celsius (J/g°C) a 25°C.
  alpha = 0.1
  u = 1
  L = 4

  for k in range (1, serie_length+1):
    partial_sum += (1/(2*k-1))*exp((-alpha*((2*k-1)**2)*(pi**2)*t)/(L**2))*sin((2*k-1)*((pi*(x-u*t))/L))

  return 0.5 - (2/pi)*partial_sum

def plot_graph ():
  x = np.linspace(-2.0, 2.0, 100)
  plt.plot(x, [analytic_solution(each_x, 1) for each_x in x], label="Exata", color="red")
  plt.xlabel('x')
  plt.ylabel('Temperatura')
  plt.title('Gráfico de Temperatura por posição')
  plt.grid(True) 
  plt.legend()
  plt.show()

def solve_tdma(size):
  # Temperaturas que aparecem na expressão do termo dj
  previous_temp = 0.0
  actual_temp = 0.0
  next_temp = 0.0

  # Expressões que calculam os elementos aj, bj, cj, e dj
  a_j = 1 + beta*((1-2*gama)*C + 2*S)
  b_j = beta*(-gama*C + S)
  c_j = beta*((1-gama)*C + S)
  d_j = (1-beta)*((1-gama)*C+S)*previous_temp + (1-(1-beta)*((1-2*gama)*C+2*S))*actual_temp - (1-beta)*(-gama*C+S)*next_temp

  a = [a_j for _ in range(size)]
  b = [b_j for _ in range(size)]
  c = [c_j for _ in range(size)]
  d = [d_j for _ in range(size)]
  x = [0.0 for _ in range(size)]
  p = [0.0 for _ in range(size)]
  q = [0.0 for _ in range(size)]

  # Condições de contorno
  p[0] = 0.0
  q[0] = T_1
  p[size-1] = 0.0
  q[size-1] = T_2

  for j in range(1, size-1):
    p[j] = b[j] / (a[j] - c[j]*p[j-1])
    q[j] = (d[j] + c[j]*q[j-1]) / (a[j] - c[j]*p[j-1])

  print(p, q)

  for j in range(size-2, -1, -1):
    print(j)
    x[j] = p[j]*x[j+1] + q[j]

  print(x)
    


    

# Variáveis iniciais

# k = 0.1 # Condutividade térmica
# p = 0.1 # Massa específica
# cp = 0.1  # Capacidade térmica

u = 1.0 # Velocidade de advecção
alpha = 0.1 # Difusividade térmica
delta_x, delta_t = 0.1, 0.1
C = (u*delta_t)/delta_x
S = (alpha*delta_t)/(delta_x**2)

beta, gama = 0.0, 0.0

T_1 = 1.0
T_2 = 0.0
L = 4.0

solve_tdma(4)


[0.0, 0.0, 0.0, 0.0] [1.0, 0.0, 0.0, 0.0]
2
1
0
[1.0, 0.0, 0.0, 0.0]
